In [ ]:
%pip install transformers sentencepiece torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import softmax
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

In [ ]:
# Define the file path for business.json
file_path = 'yelp_academic_dataset_business.json'

# Read the 'business.json' file into a DataFrame
chunks = pd.read_json(file_path, lines=True, encoding='utf-8', chunksize=10000)
business = pd.DataFrame()
for chunk in chunks:
    business = pd.concat([business, chunk])

In [ ]:
review_file_path = '/content/drive/MyDrive/Colab Notebooks/Coffee Rebrewer/yelp_dataset/yelp_academic_dataset_review.json'

# Read the 'reviews.json' file into a DataFrame
chunks = pd.read_json(review_file_path, lines=True, encoding='utf-8', chunksize=10000)
reviews = pd.DataFrame()
for chunk in chunks:
    reviews = pd.concat([reviews, chunk])

In [ ]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [ ]:
len(reviews)

6990280

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [ ]:
# drop rows that don't have categories
business = business.dropna(subset=['categories'])

# filter by city (Philadelphia)
philadelphia_businesses = business[business['city'] == 'Philadelphia']

# check for 'coffee' in the 'categories' column to identify coffee shops
philly_coffee = philadelphia_businesses[philadelphia_businesses['categories'].str.startswith('Coffee & Tea')]


In [ ]:
# define date threshold (5 years from today)
date_threshold = (datetime.now() - timedelta(days=5*365)).date()

# Ensure that the 'date' column in Reviews is a Pandas datetime object
# Convert it to date
# reviews['date'] = reviews['date'].dt.date


# Filter reviews dataset by date
recent_reviews = reviews[reviews['date'] >= date_threshold]

In [ ]:
recent_reviews

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
179241,biAKwReuwpIVMJuHIpfXNA,TReMtyEpbMRs-aSWNfd97Q,RdDS9iOb5JJhFom1Ny4xUQ,3,0,0,0,Pretty good cheesesteaks on nice fresh rolls. ...,2018-11-21
179242,1F2mkqu2F1aZtXB9nK5e2Q,QGOnRvVPwR6zgwvJU6Vciw,WW8WN8J89M3U4z1xP4MhnA,1,0,0,0,My family and I went to this location for Than...,2018-11-22
181099,hmhXtKz7c3zjMsmNR6LG3g,d2PoVe1LCtwpqBZqdE9wqA,C1D2o1VV2TDjpkG3BsdseA,1,0,0,0,By far the worst tacos I've ever eaten in my l...,2018-10-29
181345,kzHio92Wwsf1TAhH4kIlog,u534NoGOFLmACKkyQ3FR4g,6yZ2-zmRDw7sTesRn2QAug,5,0,0,0,I've been getting my nails done here for month...,2018-11-13
181347,TaouhF58T2E95-2pE5kvLg,J6Iha2b-oBD_cB8WsoICmg,yD6cXmbmhRxcd3Y84Ry5wA,5,0,0,0,She is amazing!! Had a bridesmaid dress that w...,2018-11-06
...,...,...,...,...,...,...,...,...,...
6990272,wD5ZWao_vjyT2h4xmGam8Q,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,5,1,0,1,It is very rare for a restaurant to be this go...,2022-01-17
6990273,zHZ-A1qyKDEgyZMDaD--wg,_XVdmFWSgTN6YlojUxixTA,6WaI-IN8ql0xpEKlb4q8tg,5,1,0,0,We redesigned my moms dress and mad it complet...,2022-01-17
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30


In [ ]:
# Group and count reviews by business_id
review_counts = recent_reviews['business_id'].value_counts().reset_index()
review_counts.columns = ['business_id', 'review_count']

In [ ]:
# Filter philly coffee shops with at least 25 recent reviews
philly_25_reviews = philly_coffee[philly_coffee['business_id'].isin(review_counts[review_counts['review_count'] >= 25]['business_id'])]

In [ ]:
philly_25_reviews

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3288,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,192,1,"{'RestaurantsReservations': 'False', 'Restaura...","Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."
6207,_O_KBH1MStcUkS1xk5pxsg,Franny Lou's Porch,2400 Coral St,Philadelphia,PA,19125,39.983567,-75.129386,4.5,68,1,"{'RestaurantsTakeOut': 'True', 'OutdoorSeating...","Coffee & Tea, Breakfast & Brunch, Books, Mags,...","{'Monday': '8:0-16:0', 'Tuesday': '8:0-16:0', ..."
7787,bpEiVcivZFHixcDGKXQQJA,Valerio Coffee Roasters,4373 Main St,Philadelphia,PA,19127,40.025289,-75.222787,4.0,34,1,"{'WheelchairAccessible': 'True', 'BusinessAcce...","Coffee & Tea, Bagels, Food, Breakfast & Brunch...","{'Tuesday': '8:0-14:30', 'Wednesday': '8:0-14:..."
10710,99zoELPcbwRBUqiPoaPqiQ,Bold Coffee Bar,1623 Ridge Ave,Philadelphia,PA,19130,39.969977,-75.163925,5.0,28,1,"{'RestaurantsReservations': 'False', 'OutdoorS...","Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."
25504,Q9poenJ9SS3agpPo4Pgyyg,Ray's Café & Teahouse,141 N 9th St,Philadelphia,PA,19107,39.954621,-75.154276,4.5,181,1,"{'WiFi': 'u'no'', 'RestaurantsGoodForGroups': ...","Coffee & Tea, Chinese, Restaurants, Food","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
27474,5Y7cqSgUSfLwn7ulBeO_sA,Bluestone Lane - Philadelphia,1 S Penn Sq,Philadelphia,PA,19107,39.951389,-75.163113,4.0,88,1,"{'HasTV': 'True', 'RestaurantsPriceRange2': '2...","Coffee & Tea, Breakfast & Brunch, Restaurants,...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
31528,T0qzsFxkHK9w1afnmns4fw,Konditori,40 S 17th St,Philadelphia,PA,19103,39.951838,-75.168980,4.0,102,1,"{'OutdoorSeating': 'False', 'BusinessParking':...","Coffee & Tea, Juice Bars & Smoothies, Bakeries...","{'Monday': '6:30-20:0', 'Tuesday': '6:30-20:0'..."
31620,Ku2b7oA9s56mHoflb5lJ2w,Alma Del Mar,1007 S 9th St,Philadelphia,PA,19147,39.937310,-75.158010,5.0,80,1,"{'RestaurantsTableService': 'True', 'BusinessP...","Coffee & Tea, Seafood, Restaurants, Fish & Chi...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
36643,KqUinf0QEeiZdxko2dFEsA,Shane Confectionery,110 Market St,Philadelphia,PA,19106,39.949547,-75.142698,4.5,169,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Coffee & Tea, Specialty Food, Desserts, Candy ...","{'Monday': '12:0-19:0', 'Tuesday': '11:0-22:0'..."
38129,bACIZGdHRFfNtrufH0wMbQ,New York Bagel Bakery,7555 Haverford Ave,Philadelphia,PA,19151,39.979700,-75.268085,4.5,82,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Coffee & Tea, Restaurants, Bagels, Food, Cater...","{'Monday': '0:0-0:0', 'Wednesday': '7:0-14:0',..."


In [ ]:
len(philly_25_reviews)

46

In [ ]:
# Inner join to get reviews for businesses that are in both philly_25_reviews and recent_reviews
filtered_philly_reviews = pd.merge(philly_25_reviews, recent_reviews, on='business_id', how='inner')


In [ ]:
# Group the DataFrame by 'business_id' and count the number of rows (reviews) for each group
filtered_philly_reviews['recent_review_count'] = filtered_philly_reviews.groupby('business_id')['business_id'].transform('count')


In [ ]:
#replace 'review_count' to 'recent_review_count'

filtered_philly_reviews['review_count'] = filtered_philly_reviews['recent_review_count']

filtered_philly_reviews = filtered_philly_reviews.iloc[:, :-1]

In [ ]:
filtered_philly_reviews

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",n9pdoDkIip1pKzGzbvsegQ,8cu7AJE7gyq1-SLOcxUptA,5,0,0,0,A great neighborhood cafe! I especially love t...,2019-06-14
1,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",WcGtUuiXoeJDFxGxVEbPeA,P_9jCgiS7WTBjEU6SVG9gg,5,0,0,0,this place is so hipster! i ordered crepes whi...,2018-12-10
2,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",I9Dz9urftMCFFSKgAw5a-g,Te3BwftEFN8C9WxImhfM_g,5,2,0,2,What a gem of a place in the francisville neig...,2018-11-12
3,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",PJoin-6WYD7QzBZAhQS-Iw,zX6kC-irRdPn0Qs8pcYFKQ,5,0,0,0,This is the best pancake place in a city. Cozy...,2018-11-03
4,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",iZ_f9CY8WBDb0x95tSBSFg,a8qWaac2kpRAjl_6gHwJTw,5,0,0,0,This is my favorite café to go to whenever I'm...,2019-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,"Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:30-16:0', ...",i1db72EqEjwTZzRpdgVtYA,XbZt0hG6xP8JX-k1FGgIDQ,3,2,0,0,This is a great cafe but I wish they served co...,2020-01-06
2696,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,"Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:30-16:0', ...",tGvyaaQ4NN5agP3QnbyxlQ,RnsGfdux_mwwOEIx-piakA,4,0,0,0,This is probably one of my favorite places in ...,2020-02-18
2697,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,"Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:30-16:0', ...",-lJL33FYFwZtmKtR0nJimA,QIuguXNpP_q_3y0RqHLGVw,4,3,1,3,"Love the atmosphere, feels like a place you ca...",2019-12-03
2698,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,"Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:30-16:0', ...",TUMf_xoEifSZGSKrKJEJFA,PdgD8Q6TQk4zdkOoTOuVDw,3,2,1,1,One of my go-to's when I need somewhere in Cen...,2019-06-13


In [ ]:
np.mean(filtered_philly_reviews['recent_review_count'])

112.98370370370371

In [ ]:
filtered_philly_reviews.sort_values('date')

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,hours,review_id,user_id,stars_y,useful,funny,cool,text,date,recent_review_count
1577,5asvaAT-jDbOnCM3iPlScw,ReAnimator Coffee,310 W Master St,Philadelphia,PA,19122,39.972432,-75.141024,4.5,84,...,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-15:0', '...",O2GKMON3qGomBLHZAfkbEg,NV2l8Oyvsk-hlmN4Tf82qQ,3,0,0,0,"Ordered an iced coffee, drowned it in cream an...",2018-10-27,28
779,IjMs1n7UelI7ev_5IDl5kA,Pho Saigon,"1100 S Christopher Columbus Blvd, Unit 22",Philadelphia,PA,19147,39.932705,-75.144380,4.0,419,...,"{'Monday': '9:0-21:0', 'Tuesday': '9:0-21:0', ...",e0RdNm5fOqYvD2ouLsShiA,su8U8TxjnRFfygMVcseHoA,5,0,0,0,"Overall, my favorite place to go for pho. Food...",2018-10-27,51
789,IjMs1n7UelI7ev_5IDl5kA,Pho Saigon,"1100 S Christopher Columbus Blvd, Unit 22",Philadelphia,PA,19147,39.932705,-75.144380,4.0,419,...,"{'Monday': '9:0-21:0', 'Tuesday': '9:0-21:0', ...",JVANP_JipTQQOlzowSLaYw,kT1eoBIf5PcBUOk2gVqvJg,5,0,0,0,bean there couple time the pho tastes deliciou...,2018-10-27,51
440,TUtOrI_GnVwSuHHTTEN8MA,Chhaya Cafe,1819 E Passyunk Ave,Philadelphia,PA,19148,39.927223,-75.166326,4.0,301,...,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-15:0', '...",RE-D200z4CvxBI97yiDz1A,kG36qGK61YF2JxKBzGyzaA,1,0,0,0,Food is flavorless and vastly overpriced. Avo...,2018-10-27,28
2411,9A5Gw0At6so0x-vWM0_JZw,Grindcore House,1515 S 4th St,Philadelphia,PA,19147,39.929015,-75.151736,4.5,338,...,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-16:0', '...",yl-d9RagkO5pUxSEuy1Rlw,jA8QxEykYOAwJCLJWsV24A,5,0,0,0,Love coming to this place in my neighborhood. ...,2018-10-28,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,e27sItQBZuFbBPz_Ucq76A,Tea Do,"3816 Chestnut St, Ste 36",Philadelphia,PA,19104,39.954682,-75.199947,4.0,35,...,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",NWo1U_LQR4z5guFLwGXjLA,ZuRBTpKHifS0aXplqJLvGg,5,1,0,1,Tea-Do does bubble-tea so well! I swear no oth...,2022-01-18,35
770,JrG4NINLspXPNhSXg7Q07Q,Front Street Cafe,1253 N Front St,Philadelphia,PA,19122,39.970201,-75.135634,4.0,716,...,"{'Monday': '9:0-15:0', 'Wednesday': '9:0-15:0'...",E7rmE7-I7IFDPoJNCxyh8g,0WZUbIk3Ic6njuGUYNM5uQ,2,0,0,0,"While the food is generally pretty decent, the...",2022-01-18,283
138,99zoELPcbwRBUqiPoaPqiQ,Bold Coffee Bar,1623 Ridge Ave,Philadelphia,PA,19130,39.969977,-75.163925,5.0,28,...,"{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",ldpdWC5Um4m3D8EsVRkYBg,Y_QRMsoQYldqJ_0hKsVjVw,5,0,0,0,I ordered an iced latte with caramel and their...,2022-01-19,28
2568,h_rcMQtglIiAs-Oc5d8Ozw,Old City Coffee,1136 Arch St,Philadelphia,PA,19107,39.953137,-75.159282,3.5,221,...,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:0', '...",hdmErJxaHDRO02GzqUHa1A,5eyXkKucsO2FzAuyhpRSHQ,5,0,0,0,Inside of reading terminal there's this amazin...,2022-01-19,46


In [ ]:
filtered_philly_reviews.sort_values('recent_review_count', ascending=False)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,hours,review_id,user_id,stars_y,useful,funny,cool,text,date,recent_review_count
2106,OAWa1WML2V1ZLJGD6V3nBQ,Middle Child,248 S 11th St,Philadelphia,PA,19107,39.947170,-75.159440,4.5,426,...,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-15:0', '...",Q3hjrNNHRWZIcAIn3Bosrg,CBSv-uTf6lwwiXQRoKcagg,5,1,1,1,This spot exceeded my expectations beyond word...,2021-11-23,293
2123,OAWa1WML2V1ZLJGD6V3nBQ,Middle Child,248 S 11th St,Philadelphia,PA,19107,39.947170,-75.159440,4.5,426,...,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-15:0', '...",-JqMEdSEc-KEIgbQEbRSxw,qWm8WMiW9hJedZ4p1_3miA,4,0,0,0,Great sandwiches. A little different than your...,2021-09-02,293
2125,OAWa1WML2V1ZLJGD6V3nBQ,Middle Child,248 S 11th St,Philadelphia,PA,19107,39.947170,-75.159440,4.5,426,...,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-15:0', '...",Mz15POLveKOXxDAYNB4VcQ,CCwWCzNXuLRqzOcGDJS_Yw,5,0,0,0,Absolutely LOVED this place! Went here on a da...,2019-06-25,293
2126,OAWa1WML2V1ZLJGD6V3nBQ,Middle Child,248 S 11th St,Philadelphia,PA,19107,39.947170,-75.159440,4.5,426,...,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-15:0', '...",Zu7J4zNiAQMsPHdrGCa0gg,rO9ojBDK8lw0x6QVoa4DbA,4,2,0,2,So I realize that it's a little ridiculous to ...,2018-11-12,293
2127,OAWa1WML2V1ZLJGD6V3nBQ,Middle Child,248 S 11th St,Philadelphia,PA,19107,39.947170,-75.159440,4.5,426,...,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-15:0', '...",gbC4TyMXctuEuYjHoemyVg,MpSBscSS0jTmJNYdBdXzrQ,5,0,0,0,Very tasty sandwich despite being pricy. Would...,2020-08-25,293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,QTjtz-bfm8Yrrv6t0PP0FQ,Knockbox Cafe,405 S 45th St,Philadelphia,PA,19104,39.951443,-75.212341,4.0,115,...,"{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",05llmFHuwgg9XdlZE9XmjA,j1uGBey6CkyntcjlkpOq8g,5,3,1,1,Awesome place. Have to say I have neglected th...,2020-11-13,25
1312,QTjtz-bfm8Yrrv6t0PP0FQ,Knockbox Cafe,405 S 45th St,Philadelphia,PA,19104,39.951443,-75.212341,4.0,115,...,"{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",wtO0sAmLeFuApxt_qyZfnw,qpHcaQpmRVbngq1-7n_ViA,4,3,0,2,Sweet West Philly cafe. Since I've lived in th...,2020-10-18,25
1311,QTjtz-bfm8Yrrv6t0PP0FQ,Knockbox Cafe,405 S 45th St,Philadelphia,PA,19104,39.951443,-75.212341,4.0,115,...,"{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",1GxGTqCNM0mlVKFWtawwOg,3jD3l6AaI3cMuQ8OA2EFQw,3,0,1,0,"I was greeted by ""Honey"" and then given ""Sweet...",2020-10-03,25
1310,QTjtz-bfm8Yrrv6t0PP0FQ,Knockbox Cafe,405 S 45th St,Philadelphia,PA,19104,39.951443,-75.212341,4.0,115,...,"{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",7guqNeXXtpc5Q69kFScUPQ,ipOgF1WDDQIbwhZwQ77Atg,5,0,0,0,"Oh my God, Knockbox Cafe was certainly fabulou...",2021-01-06,25


In [ ]:
filtered_philly_reviews.to_csv('filtered_philly_reviews.csv')